# CONDOR usage example
Author: 
Genís Calderer<sup>1</sup>. 

<sup>1</sup>[Kuijjer Lab](https://www.kuijjerlab.org/), NCMM, Oslo, Norway. - genis.calderer@gmail.com

## Introduction
The CONDOR<sup>1</sup> method is an implementation of the brim algorithm<sup>2</sup> for the analysis of bipartite networks. The purpose of this algorithm is to find a community structure in bipartite networks that takes into account the bipartite structure of the network as opposed to using the network as if it did not have an extra structure.
This algorithm was first described in the paper "Modularity and community detection in bipartite networks" by Michael J. Barber<sup>2</sup>. The python implementation of CONDOR is based on the R version presented in the paper "Bipartite Community Structure of eQTLs" by John Platig , Peter J. Castaldi, Dawn DeMeo, John Quackenbush<sup>2</sup>.

This guide will show how to use CONDOR using a toy network of pollinization between bee species and flower species. It is a small network but as we will see it has a quite well defined modularity structure.

This vignette can be ran on netbooks server or locally by setting the `runserver` parameter

In [ ]:
runserver=1

## 1. Importing CONDOR from netZooPy

On the server, we need to change the working directory to the `data` folder of the current useer.

In [ ]:
if runserver==1:
    import os
    os.chdir('../data/')
    ppath='/opt/data/'

In order to use the CONDOR, the functions have to be imported from netZooPy as follows:

In [ ]:
from netZooPy import condor  # To load condor
import pandas as pd          # To read dataframe

To check the parameters and information on the main condor functions type this:

In [ ]:
help(condor.condor_object)

## 2. Loading the network into a CONDOR object

To use the CONDOR method we first have to import a network's edgelist into a pandas dataframe.

In [ ]:
network = pd.read_csv(ppath+"toynetwork.csv",index_col=0)
network.head(5)

The network is a data frame that has three columns: 1) a source node (pollinators), 2) a target node (plants), and 3) interaction column that represents the edge weights in the network.

We initialize the CONDOR object with the `condor_object` function:

In [ ]:
condor_object = condor.condor_object(dataframe=network)

The condor object contains several features associated to the network and once initialized is passed to the other condor functions.

The condor object contains empty slot that will be populated once the analysis is done. `G` contains the graph, `tar_names` is the name of the target nodes, `reg_names` is the name of the source nodes, `edges` contains the edges between nodes in the bipartite graph, `modularity` wil host the modularity score of the graph, `reg_memb` is the community assignement of each target node, and `Qcomms` is the modularity scores for each commuity.

### 2.1. Create a CONDOR object from an edgelist in a file

A CONDOR object can be created from an edgelist encoded into a file as following:

In [ ]:
condor_object = condor.condor_object(network_file=ppath+"toynetwork.csv")

By default the method uses csv edgelists with header and index colum, consult the function documentation for details about modifying the format.

### 2.2. Create a CONDOR object from a DataFrame

A CONDOR object can also be created from a pandas DataFrame. We initialize the CONDOR object with the parameter `dataframe=network`

In [ ]:
condor_object = condor.condor_object(dataframe=network)

## 3. Running CONDOR

The next step is computing the initial community structure. By default we use the Louvain<sup>3</sup> method. Consult the documentation of the method for different initial community structure algorithms and projection.

In [ ]:
condor_object.initial_community()

This network has a modularity score of 0.52, which indicates a rich community structure. The condor object now has a community structure associated but it is not specific for bipartite networks. We apply the brim<sup>2</sup> algorithm to find the bipartite community structure. The condor object now has a community structure associated but it is not specific for bipartite networks. We apply the brim algorithm to find the bipartite community structure.

In [ ]:
condor_object.brim(deltaQmin="def")

The numbers in the output of this function show the bipartite modularity score for each iteration. The modularity of a bipartite network is a value from 0 to 1 that quantifies how well separated is the network into modules. A score of 0.52 is quite high.

## 4. Results

The resulting condor_object of the above process has the membership of the target and regulator nodes into the different communities that have been found. These are stored in the variables `tar_memb` and `reg_memb`.

For example if we want to see the membership of the $reg$ nodes we can do it as follows:

In [ ]:
condor_object["reg_memb"].head(5)

`Andromeda.glaucophylla` belongs to community 3, while `Aronia.melanocarpa` belongs to community 4.

## 5. Running CONDOR from filename

We note that the guide above shows how to use the method step by step. There is also the possibility to run automatically the whole process starting only with the filename of the network's edgelist in csv format.
This is done using the `condor` function, and it outputs the target and regulator memberships into csv files. This however allows less control on the parameters of the method.

In [ ]:
condor.run_condor(ppath+"toynetwork.csv")

We see that we obtain the same modularity score of 0.52 as in the previous steps.

Check the documentation of the `run_condor` function to toggle the options of the method.

In [ ]:
help(condor.run_condor)

# References
1- Platig, John, et al. "Bipartite community structure of eQTLs." PLoS computational biology 12.9 (2016): e1005033.

2- Barber, Michael J. "Modularity and community detection in bipartite networks." Physical Review E 76.6 (2007): 066102.

3- Blondel, Vincent D., et al. "Fast unfolding of communities in large networks." Journal of statistical mechanics: theory and experiment 2008.10 (2008): P10008.